# Pre processing
Pre processing structured as a function for ease.

In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import pyxem as pxm
from pathlib import Path
from skimage import filters

## Loading in the unprocessed data

In [3]:



# datapath = Path("D:/Project_thesis/Code_data/Sample_data/20230616_130045.zspy")
datapath = Path("./small_sample/tilt1_small.zspy")
print(f'Loading dataset {datapath.absolute()}')
dataset = hs.load(str(datapath), lazy=False)
dataset

Loading dataset c:\Users\KajaEA\Project_assignment\Code\Template_matching2\small_sample\tilt1_small.zspy


<ElectronDiffraction2D, title: , dimensions: (128, 67|256, 256)>

## Centering

In [4]:
def make_nav_mask(signal, width=None):
    mask = hs.signals.Signal2D(np.zeros(signal.axes_manager.navigation_shape, dtype=bool).T).T
    if width is not None:
        mask.inav[width:-width, width:-width] = True
    return mask

def correct_shifts_COM(signal, com_mask, nav_mask=None, plot_results=False, inplace=False):
    com = signal.center_of_mass(mask=com_mask)
    if plot_results:
        com.get_bivariate_histogram().plot()
        
    beam_shift = pxm.signals.BeamShift(com.T)
    beam_shift.make_linear_plane(mask=nav_mask)
        
    x_shift, y_shift = [beam_shift.isig[ax] - signal.axes_manager.signal_shape[ax]/2.0 for ax in (0, 1)]
    
    print(f'Estimated beam shift X min/max = ({x_shift.min().data}, {x_shift.max().data})\nEstimated beam shift Y min/max = ({y_shift.min().data}, {y_shift.max().data})')
    
    dp_max_before = signal.max(axis=[0, 1])
    
    #A trick to make sure that the shifted signal contains the same metadata etc as the original signal. Might not be needed...
    if not inplace:
        shifted_signal = signal.deepcopy()
    else:
        shifted_signal = signal
    
    shifted_signal.shift_diffraction(x_shift, y_shift, inplace=True)
    
    dp_max_after = shifted_signal.max(axis=[0, 1])
        
    if plot_results:
        hs.plot.plot_images([dp_max_before, dp_max_after], overlay=True, colors=['w', 'r'], axes_decor='off', alphas=[1, 0.75])
    
    return shifted_signal, x_shift, y_shift

In [5]:
dp_max = dataset.max(axis=[0, 1])

dp_max.plot(norm='symlog')
x0, y0 = dp_max.axes_manager.signal_shape
x0, y0 = x0//2, y0//2
roi = hs.roi.CircleROI(x0, y0, 10)
roi.add_widget(dp_max)

c:\Users\KajaEA\miniconda3\envs\pyxem\Lib\site-packages\ipykernel\eventloops.py:128: UserWarning: All values for SymLogScale are below linthresh, making it effectively linear. You likely should lower the value of linthresh. 
  el.exec() if hasattr(el, 'exec') else el.exec_()


In [6]:

shifted_signal, shift_x, shift_y = correct_shifts_COM(dataset, (roi.cx, roi.cy, roi.r), plot_results=True)
# If we want to center another time!
nx, ny = shifted_signal.axes_manager.signal_shape
nx, ny = nx/2.0, ny/2.0
shifted_signal_2, shift_x, shift_y = correct_shifts_COM(shifted_signal, (nx, ny, roi.r,), plot_results=True)
dataset = shifted_signal_2

[########################################] | 100% Completed | 122.76 s
Estimated beam shift X min/max = ([-3.14599124], [-1.15347152])
Estimated beam shift Y min/max = ([-3.28736481], [-1.74927573])
[########################################] | 100% Completed | 152.65 s
[########################################] | 100% Completed | 112.86 s
Estimated beam shift X min/max = ([-0.12286907], [0.09448368])
Estimated beam shift Y min/max = ([-0.16089316], [0.04410607])
[########################################] | 100% Completed | 157.21 s


In [7]:
scale = 0.009451434347767504
dataset.set_diffraction_calibration(scale)

In [8]:
dataset.save("D:/Project_thesis/Code_data/Sample_data/20230616_130045_centered.zspy")

## Crop data

In [ ]:
dataset.plot(norm = 'symlog')
roi = hs.roi.RectangularROI(left=89.62, right=219.0, top=20.94, bottom=113.0) # Must change for each tilt
roi.add_widget(dataset)

In [ ]:
dataset = roi(dataset)

## Processing
Function for pre processing already centered data

In [4]:
def set_threshold_below(signal,threshold,background_value):
    signal.data = np.where(signal.data<threshold,background_value,signal.data)
    return signal.data

def set_threshold_above(signal,threshold,background_value):
    signal.data = np.where(signal.data>threshold,background_value,signal.data)
    return signal.data

def pre_processing(centered_data):
    # Normalizing the data
    centered_data = centered_data / centered_data.max(axis=[0, 1, 2, 3]).data
    centered_data.data.dtype
    # Background subtraction
    method = 'difference of gaussians'
    min_sigma = 5 #2.0  #3.2 used before
    max_sigma = 9 #6.0  # used 12.0 before
    centered_data = centered_data.subtract_diffraction_background(method=method, min_sigma=min_sigma,
                                                    max_sigma=max_sigma, lazy_result=True)
    centered_data.metadata.add_dictionary(
        {'Preprocessing': 
            {'Background': 
                {'method': method,
                'min_sigma': min_sigma,
                'max_sigma': max_sigma}
            }
        }
    )
    # Masking the center spot
    len_data = np.shape(centered_data.data)[-1]
    center_mask = np.ones((len_data,len_data))
    center_mask[int(len_data/2)-8:int(len_data/2)+8,int(len_data/2)-8:int(len_data/2)+8]= 0
    centered_data.data *= center_mask
    # Gaussian smoothing
    smoothing_sigma = 0.5
    centered_data.map(filters.gaussian, sigma=smoothing_sigma, inplace=True)
    centered_data.metadata.add_dictionary(
        {'Preprocessing': 
            {'Smoothing': 
                {'method': "filters.gaussian",
                'sigma': smoothing_sigma}
            }
        }
    )
    # Thresholding to remove
    set_threshold_above(centered_data,0.01,0.01) # Upper limit
    set_threshold_below(centered_data,0.0006,0) # Lower limit

    return centered_data


## Process the dataset using the function above

In [ ]:
# Loaf in 
datapath = Path("D:/Project_thesis/Code_data/Sample_data/20230616_130045_centered.zspy")
# datapath = Path("./Midway_data/image_calibrated_centered.zspy")
print(f'Loading dataset {datapath.absolute()}')
dataset = hs.load(str(datapath), lazy=False)
dataset

In [5]:
processed_data = pre_processing(dataset)

c:\Users\KajaEA\miniconda3\envs\pyxem\Lib\site-packages\pyxem\signals\diffraction2d.py:383: VisibleDeprecationWarning: Argument `lazy_result` is deprecated and will be removed in version 1.0.0. To avoid this warning, please do not use `lazy_result`. Use `lazy_output` instead. See the documentation of `subtract_diffraction_background()` for more details.
  name="lazy_result", alternative="lazy_output", since="0.15.0", removal="1.0.0"


In [6]:
# processed_data.save('d:/Project_thesis/Code_data/Sample_data_post/20231010_192356_processed_cropped_v1.zspy')
processed_data.save('./small_sample/tilt1_small_processed.zspy')

In [ ]:
# datapath = Path("D:/Project_thesis/Code_data/Sample_data_post/20231010_192356.zspy")
datapath = Path("./Midway_data/processed_image_v6.zspy")
print(f'Loading dataset {datapath.absolute()}')
processed_data = hs.load(str(datapath), lazy=False)
processed_data

In [ ]:
scale = 0.009451434347767504
processed_data.set_diffraction_calibration(scale)
processed_data.set_scan_calibration(scale)

dataset.set_diffraction_calibration(scale)
dataset.set_scan_calibration(scale)

In [ ]:
# Plot pre and post processed data:
hs.plot.plot_signals([dataset,processed_data],cmap = 'magma_r',norm = 'symlog',title=' ')


In [ ]:
processed_data.plot(cmap='magma_r')